In [287]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [288]:
#Get All Years into Single DataFrame
import glob
import os
path = os.getcwd() + '/AllYears/'
allFiles = glob.glob(path + "/*.csv")

In [289]:
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    
    #some years have tabs at end of column names
    df.columns = [i.strip('\n') for i in df.columns]
    
    
    #Make sure SALE DATE column is a Date Type
    df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])
    
    #create year column
    df['YEAR'] = df['SALE DATE'].apply(lambda r: r.year)
    list_.append(df)
rawdf = pd.concat(list_,)

In [290]:
rawdf.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
0,0,2,BATHGATE,01 ONE FAMILY HOMES,1,2908,25,,A1,460 E176TH STREET,,10457,1,0,1,2580,2070,1910,1,A1,0,2008-11-06,2008
1,1,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,28,,A1,2329 WASHINGTON AVENUE,,10458,1,0,1,1103,1290,1910,1,A1,370000,2008-05-27,2008
2,2,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,29,,A9,2327 WASHINGTON AVE,,10458,1,0,1,1095,1260,1910,1,A9,360000,2008-04-18,2008
3,3,2,BATHGATE,01 ONE FAMILY HOMES,1,3043,56,,A1,1950 BATHGATE AVENUE,,10457,1,0,1,2356,2048,1901,1,A1,0,2008-09-03,2008
4,4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,,A1,2085 BATHGATE,,10457,1,0,1,2060,1629,1899,1,A1,274764,2008-08-04,2008


In [291]:
#cleanup DataFrame
#columns to keep
keep =(
[u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
 u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u'SALE PRICE', u'SALE DATE', 'YEAR'])

df = rawdf[keep].reset_index(drop=True)

df = df[df['SALE PRICE'] > 1000]

In [292]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
1,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,28,A1,2329 WASHINGTON AVENUE,10458,1,0,1,1103,1290,1910,1,A1,370000,2008-05-27,2008
2,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,29,A9,2327 WASHINGTON AVE,10458,1,0,1,1095,1260,1910,1,A9,360000,2008-04-18,2008
4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE,10457,1,0,1,2060,1629,1899,1,A1,274764,2008-08-04,2008
5,2,BATHGATE,01 ONE FAMILY HOMES,1,3047,11,A1,482 EAST 181 STREET,10457,1,0,1,1556,1240,1901,1,A1,283000,2008-06-11,2008
6,2,BATHGATE,01 ONE FAMILY HOMES,1,3053,105,A1,2323 BASSFORD AVENUE,10458,1,0,1,922,1248,1901,1,A1,250000,2008-08-04,2008


In [293]:
df.count()

BOROUGH                           376130
NEIGHBORHOOD                      376130
BUILDING CLASS CATEGORY           376130
TAX CLASS AT PRESENT              376130
BLOCK                             376130
LOT                               376130
BUILDING CLASS AT PRESENT         376130
ADDRESS                           376130
ZIP CODE                          376130
RESIDENTIAL UNITS                 376130
COMMERCIAL UNITS                  376130
TOTAL UNITS                       376130
LAND SQUARE FEET                  376130
GROSS SQUARE FEET                 376130
YEAR BUILT                        376130
TAX CLASS AT TIME OF SALE         376130
BUILDING CLASS AT TIME OF SALE    376130
SALE PRICE                        376130
SALE DATE                         376130
YEAR                              376130
dtype: int64

In [295]:
#Only keep Zip Codes with data from all 4 Years
Num_Years = 7
EveryYear = (df.groupby('ZIP CODE')['YEAR'].nunique() == Num_Years)
zips = EveryYear[EveryYear == True].index

#Only keep Zip Codes with over 500 sales in the last 4 years
threshold = 500
Enough_Sales = (df.groupby('ZIP CODE').count() > threshold)
zips2 = groups[groups['BLOCK'] == True].index
print len(zips), len(zips2)

179 154


In [296]:
#Find intersection
keep_zips = np.intersect1d(zips,zips2)

In [365]:
clean_df = df[df['ZIP CODE'].isin(keep_zips)].reset_index(drop= True)

In [343]:
clean_df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
0,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,28,A1,2329 WASHINGTON AVENUE,10458,1,0,1,1103,1290,1910,1,A1,370000,2008-05-27,2008
1,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,29,A9,2327 WASHINGTON AVE,10458,1,0,1,1095,1260,1910,1,A9,360000,2008-04-18,2008
2,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE,10457,1,0,1,2060,1629,1899,1,A1,274764,2008-08-04,2008
3,2,BATHGATE,01 ONE FAMILY HOMES,1,3047,11,A1,482 EAST 181 STREET,10457,1,0,1,1556,1240,1901,1,A1,283000,2008-06-11,2008
4,2,BATHGATE,01 ONE FAMILY HOMES,1,3053,105,A1,2323 BASSFORD AVENUE,10458,1,0,1,922,1248,1901,1,A1,250000,2008-08-04,2008


In [344]:
#Add Features
#clean_df.groupby(['ZIP CODE', 'YEAR'])['SALE PRICE'].mean()
#clean_df.groupby(['ZIP CODE', 'YEAR'])['SALE PRICE'].std()
#clean_df.groupby(['ZIP CODE', 'YEAR'])['SALE PRICE'].count()
#clean_df.groupby(['ZIP CODE', 'YEAR'])['SALE PRICE'].median()

ZIP CODE  YEAR
10001     2008     6524194
          2009     4663023
          2010     6215075
          2011     8329433
          2012     4853504
          2013     7249010
          2014    10840595
10002     2008     1354956
          2009      674625
          2010     1433065
          2011     1214870
          2012     1750014
          2013     1447769
          2014     1859802
10003     2008     1915623
          2009     1264756
          2010     2509130
          2011     1801484
          2012     1667837
          2013     2258257
          2014     1905040
10005     2008     1579375
          2009     1927024
          2010     1847454
          2011     4582687
          2012     3833599
          2013     6155495
          2014     1550603
10007     2008     2668722
          2009     2537360
                    ...   
11435     2013      392509
          2014      406272
11436     2008      371210
          2009      300644
          2010      277081
          201

Now, we add some historical features to the DataFrame. This includes the means, standard deviation, number and median for the sales prices per zip code.

In [ ]:
x = clean_df.groupby(['ZIP CODE', 'YEAR'])['SALE PRICE']
means = x.mean().to_dict()
stds = x.std().to_dict()
counts = x.count().to_dict()
medians = x.median().to_dict()

for i in xrange(1,4):
    clean_df[str(i)+'-mean'] = (clean_df[clean_df.YEAR > 2010].
                           apply(lambda r: means[(r['ZIP CODE'],r['YEAR']-i)], axis = 1)
                           )
    clean_df[str(i)+'-std'] = (clean_df[clean_df.YEAR > 2010].
                           apply(lambda r: stds[(r['ZIP CODE'],r['YEAR']-i)], axis = 1)
                           )
    clean_df[str(i)+'-count'] = (clean_df[clean_df.YEAR > 2010].
                           apply(lambda r: counts[(r['ZIP CODE'],r['YEAR']-i)], axis = 1)
                           )
    clean_df[str(i) + '-median'] =(clean_df[clean_df.YEAR > 2010].
                           apply(lambda r: medians[(r['ZIP CODE'],r['YEAR']-i)], axis = 1)
                           )

In [392]:
clean_df[(clean_df['YEAR'] > 2010)]

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR,1-mean,2-mean,3-mean,1-std,1-count,2-std,2-count,3-std,3-count
149205,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,A1,467 EAST 185 STREET,10458,1,0,1,1667,1296,1910,1,A1,329000,2011-01-19,2011,1111170,592967,1261909,1500785.849714,85,802848.709267,101,2014719.970026,135
149206,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE AV,10457,1,0,1,2060,1629,1899,1,A1,288500,2011-06-29,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149207,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,28,A1,540 EAST 182ND STREET,10457,1,0,1,1209,1048,1901,1,A1,122500,2011-03-21,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149208,2,BATHGATE,02 TWO FAMILY HOMES,1,2929,128,B1,559 CLAREMONT PARKWAY,10457,2,0,2,2361,2394,1995,1,B1,365000,2011-07-13,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149209,2,BATHGATE,02 TWO FAMILY HOMES,1,3030,76,B2,4435 PARK AVENUE,10457,2,0,2,2444,2125,1899,1,B2,375000,2011-05-18,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149210,2,BATHGATE,02 TWO FAMILY HOMES,1,3036,43,B1,2057 WASHINGTON AVENUE,10457,2,0,2,3323,3438,1997,1,B1,360000,2011-06-29,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149211,2,BATHGATE,02 TWO FAMILY HOMES,1,3037,6,B2,4392 PARK AVENUE,10457,2,0,2,2275,3240,1899,1,B2,115000,2011-01-29,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149212,2,BATHGATE,02 TWO FAMILY HOMES,1,3039,65,B2,465 E.185TH STREET,10458,2,0,2,1667,1296,1910,1,B2,135000,2011-07-19,2011,1111170,592967,1261909,1500785.849714,85,802848.709267,101,2014719.970026,135
149213,2,BATHGATE,02 TWO FAMILY HOMES,1,3046,44,B1,2067 BATHGATE AVENUE,10457,2,0,2,1417,2995,2009,1,B1,390486,2011-07-08,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125
149214,2,BATHGATE,02 TWO FAMILY HOMES,1,3046,45,B1,2065 BATHGATE AVENUE,10457,2,0,2,1417,2995,2009,1,B1,380250,2011-03-29,2011,1669179,564211,963242,3796324.238692,104,898901.892011,100,1874234.472546,125


In [399]:
clean_df.groupby(['ZIP CODE', 'YEAR'])['SALE PRICE'].median()

ZIP CODE  YEAR
10001     2008    1333569.0
          2009     980000.0
          2010    1367500.0
          2011    1263723.5
          2012    1265000.0
          2013    1205000.0
          2014    1200000.0
10002     2008     587895.0
          2009     499990.0
          2010     683079.5
          2011     573000.0
          2012     553750.0
          2013     560037.0
          2014     660000.0
10003     2008     895000.0
          2009     731250.0
          2010     992500.0
          2011     830000.0
          2012     852275.0
          2013     867500.0
          2014    1148320.0
10005     2008     972428.0
          2009     916425.0
          2010     913750.0
          2011     916577.5
          2012     968623.0
          2013    1027500.0
          2014    1075000.0
10007     2008    2367431.0
          2009    2418343.0
                    ...    
11435     2013     255000.0
          2014     290000.0
11436     2008     360918.5
          2009     291500.0
     